In [1]:
import tensorflow as tf
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [2]:
digits = load_digits()
X = digits.data
y = digits.target
y = LabelBinarizer().fit_transform(y)
X_train , X_test, y_train, y_test = train_test_split(X,y,test_size=.3)

In [3]:
def add_layer(inputs,in_size,out_size,n_layer,activation_function=None):
    layer_name = "layer%s" % n_layer
    with tf.name_scope('layer'):
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.random_normal([in_size,out_size],name='w'))
            tf.summary.histogram(layer_name+"/weights",Weights)
        with tf.name_scope('bias'):
            biases = tf.Variable(tf.zeros([1,out_size])+0.1,name='b')
            tf.summary.histogram(layer_name+"/bias",biases)
        Wx_plus_b = tf.matmul(inputs,Weights) + biases
        Wx_plus_b = tf.nn.dropout(Wx_plus_b,keep_prob)
        if(activation_function is None):
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b)
        tf.summary.histogram(layer_name + '/outputs', outputs)
    return outputs

In [4]:
keep_prob = tf.placeholder(tf.float32)
xs = tf.placeholder(tf.float32, [None, 64])  # 8x8
ys = tf.placeholder(tf.float32, [None, 10])

In [5]:
# add output layer
l1 = add_layer(xs, 64, 50, 1, activation_function=tf.nn.tanh)
l2 = add_layer(l1, 50, 50, 1, activation_function=tf.nn.tanh)
prediction = add_layer(l2, 50, 10, 2, activation_function=tf.nn.softmax)

In [6]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),reduction_indices=[1]))  # loss
tf.summary.scalar('loss', cross_entropy)
train_step = tf.train.AdamOptimizer(0.5).minimize(cross_entropy)

In [7]:
sess = tf.Session()
merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter("logs/train",sess.graph)
test_writer = tf.summary.FileWriter("logs/test",sess.graph)
init = tf.global_variables_initializer()

In [10]:
def compute_accuracy(v_xs,v_ys,prob):
    global prediction
    y_pre = sess.run(prediction,feed_dict={xs:v_xs,keep_prob:prob})
    correct_prediction = tf.equal(tf.argmax(y_pre,1),tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    result = sess.run(accuracy,feed_dict={xs: v_xs, ys: v_ys,keep_prob:prob})
    return result

In [11]:
sess.run(init)
for i in range(2000):
    sess.run(train_step,feed_dict={xs:X_train,ys:y_train,keep_prob:0.6})
    if(i%20==0):
#         train_result = sess.run(merged, feed_dict={xs: X_train, ys: y_train, keep_prob: 1})
#         test_result = sess.run(merged, feed_dict={xs: X_test, ys: y_test, keep_prob: 1})
#         train_writer.add_summary(train_result, i)
#         test_writer.add_summary(test_result, i)
        res = compute_accuracy(X_test,y_test,1)
        print(res)

0.107407
0.353704
0.448148
0.544444
0.609259
0.607407
0.675926
0.707407
0.711111
0.722222
0.751852
0.744444
0.768519
0.792593
0.781482
0.77963
0.77963
0.814815
0.790741
0.8
0.805556
0.792593
0.812963
0.824074
0.805556
0.82037
0.82963
0.835185
0.82037
0.844444
0.837037
0.827778
0.840741
0.833333
0.851852
0.837037
0.851852
0.848148
0.857407
0.85
0.861111
0.862963
0.87037
0.874074
0.87037
0.861111
0.851852
0.881481
0.862963
0.864815
0.87963
0.87037
0.881481
0.885185
0.883333
0.887037
0.883333
0.887037
0.9
0.896296
0.885185
0.892593
0.905556
0.887037
0.901852
0.892593
0.901852
0.896296
0.894444
0.9
0.894444
0.890741
0.9
0.898148
0.901852
0.907407
0.896296
0.901852
0.901852
0.9
0.903704
0.901852
0.907407
0.901852
0.911111
0.905556
0.905556
0.909259
0.911111
0.905556
0.907407
0.905556
0.907407
0.907407
0.901852
0.907407
0.912963
0.907407
0.907407
0.909259
